In [3]:
import random
from random import shuffle

from PIL import Image, ImageFilter
from IPython import display

from matplotlib.pyplot import imshow
%matplotlib inline

import pickle
import time
import error_table

from sampling_utils import *
from gen_utils import *
from evaluation_utils import *
from config_db import ConfigDB



In [51]:
num_pics = 500
min_num_cars = 1
max_num_cars = 4

domains = [[52,87],[1,37],[0,1],[0.35,1],[0.5,1.5],[0.25,1],[0.5,1],[0.5,1]]
types = ['int','int','float','float','float','float','float','float']

gen_misclass = True
store_config_db = False
store_err_tab = True

IOMIN_THRESH = 0.4
IOU_THRESH = 0.5
AP_THRESH = 0.75
REC_THRESH = 0.75
NN_PROB_THRESH = 0.5

if gen_misclass:
    checkpoint = '/home/tommaso/analyzeNN/data/train_0/checkpoint/train/model.ckpt-5000'
    import squeezedet as nn
    net = nn.init(checkpoint)

INFO:tensorflow:Summary name mean iou is illegal; using mean_iou instead.
INFO:tensorflow:Restoring parameters from /home/tommaso/analyzeNN/data/train_0/checkpoint/train/model.ckpt-5000


In [4]:
for j in range(min_num_cars,max_num_cars):
    
    total_attempts = 0
    rejections = 0
    
    # Init data structure
    if store_err_tab:
        err_table = error_table()
    if store_config_db:
        config_db = ConfigDB()
    
    path_data_set = './data/train_0/mis/'
    #path_data_set = '/home/tommaso/analyzeNN/data/train_0/mis/mis_' + str(j) + '/'
    
    i = 0
    v = 0

    while i < num_pics:

        # Sample configuration
        #num_cars = random.randint(min_num_cars, max_num_cars)
        num_cars = j
        sample_dims = [1,num_cars,num_cars,num_cars,1,1,1,1]
        sample, norm_sample = get_sample(v, 'random', types=types, domains=domains, dims=sample_dims)
        sample[3].sort(reverse=True)
        norm_sample[3].sort(reverse=True)
        
        total_attempts += 1
        
        good_sample = True
        # Check sample distance from stored ones
        if store_config_db:
            sample_pad = pad_sample(sample)
            good_sample = config_db.eps_dist(sample_pad,0.5)
            
            
        if good_sample:
            # Generate image from sample
            img, ground_boxes, img_info = gen_image(sample)

            # Is it a good picture? (cars don't overlap too much)
            save_img = True
            for m in range(len(ground_boxes)):
                for n in range(m+1, len(ground_boxes)):
                    if iomin(ground_boxes[m], ground_boxes[n]) > IOMIN_THRESH:
                        save_img = False

            ap = 0
            recall = 0
                        
            # Check misclassification
            if gen_misclass and save_img:
                # Save tmp image    
                tmp_img_file_name = './tmp.png'
                img.save(tmp_img_file_name)

                # Classify img
                (boxes,probs,cats) = nn.classify(tmp_img_file_name, net, NN_PROB_THRESH) 

                # Not cars
                if any(cats):
                    save_img = True
                else:
                    ap, recall = average_precision(ground_boxes, boxes, IOU_THRESH)
                    save_img =  (ap <= AP_THRESH or recall <= REC_THRESH)
                        
                        

            if save_img:
                
                if store_config_db:
                    config_db.add(sample_pad)            
                
                file_name = 'm_0_' + str(j) + '_' + str(i).zfill(6)
                
                print(file_name + ' ' + str(ap) + ' ' + str(recall))
                
                # Save data
                save_image(img, file_name, path_data_set)
                save_label(ground_boxes, file_name, path_data_set)
                
                # Save in image set
                with open(path_data_set + 'image_set.txt', "a+") as im_set:
                    im_set.write(file_name + '\n')
                    im_set.close()
                
                if store_err_tab:
                    err_table.update_with_elem(img_info)
                    
                i+=1
        else:
            rejections += 1

#         if(total_attempts %50 == 0):
#             print('gen pics: ' + str(total_attempts) + ' | misclass pics: ' + str(i) + ' | rejections: ' +str(rejections))
            
print('END gen pics: ' + str(total_attempts) + ' | misclass pics: ' + str(i) + ' | rejections: ' +str(rejections))

if store_err_tab:
    err_table_file = open(path_data_set + 'error_table' + str(),'w')
    pickle.dump(err_table,err_table_file)
    err_table_file.close()

m_0_1_000000 0.5 1.0
m_0_1_000001 0.0 0.0
m_0_1_000002 0.5 1.0
m_0_1_000003 0.0 0.0
m_0_1_000004 0.0 0.0
m_0_1_000005 0.0 0.0
m_0_1_000006 0.0 0.0
m_0_1_000007 0.5 1.0
m_0_1_000008 0.0 0.0
m_0_1_000009 0.0 0.0
m_0_1_000010 0.0 0.0
m_0_1_000011 0.333333333333 1.0
m_0_1_000012 0.5 1.0
m_0_1_000013 0.5 1.0
m_0_1_000014 0 0
m_0_1_000015 0 0
m_0_1_000016 0.5 1.0
m_0_1_000017 0.0 0.0
m_0_1_000018 0.5 1.0
m_0_1_000019 0.5 1.0
m_0_1_000020 0.0 0.0
m_0_1_000021 0.5 1.0
m_0_1_000022 0.0 0.0
m_0_1_000023 0.5 1.0
m_0_1_000024 0.0 0.0
m_0_1_000025 0.0 0.0
m_0_1_000026 0 0
m_0_1_000027 0.0 0.0
m_0_1_000028 0 0
m_0_1_000029 0.5 1.0
m_0_1_000030 0 0
m_0_1_000031 0 0
m_0_1_000032 0.0 0.0
m_0_1_000033 0.5 1.0
m_0_1_000034 0.0 0.0
m_0_1_000035 0.5 1.0
m_0_1_000036 0.5 1.0
m_0_1_000037 0.0 0.0
m_0_1_000038 0.5 1.0
m_0_1_000039 0.0 0.0
m_0_1_000040 0.0 0.0
m_0_1_000041 0.5 1.0
m_0_1_000042 0.5 1.0
m_0_1_000043 0.0 0.0
m_0_1_000044 0.0 0.0
m_0_1_000045 0.0 0.0
m_0_1_000046 0.333333333333 1.0
m_0_1_000047 0.

m_0_1_000388 0.0 0.0
m_0_1_000389 0.5 1.0
m_0_1_000390 0.5 1.0
m_0_1_000391 0.5 1.0
m_0_1_000392 0.0 0.0
m_0_1_000393 0.0 0.0
m_0_1_000394 0.0 0.0
m_0_1_000395 0.5 1.0
m_0_1_000396 0.5 1.0
m_0_1_000397 0.5 1.0
m_0_1_000398 0.333333333333 1.0
m_0_1_000399 0.0 0.0
m_0_1_000400 0.0 0.0
m_0_1_000401 0.0 0.0
m_0_1_000402 0.5 1.0
m_0_1_000403 0.5 1.0
m_0_1_000404 0.5 1.0
m_0_1_000405 0.0 0.0
m_0_1_000406 0.0 0.0
m_0_1_000407 0.5 1.0
m_0_1_000408 0.5 1.0
m_0_1_000409 0.0 0.0
m_0_1_000410 0.5 1.0
m_0_1_000411 0.333333333333 1.0
m_0_1_000412 0.5 1.0
m_0_1_000413 0.0 0.0
m_0_1_000414 0.0 0.0
m_0_1_000415 0.0 0.0
m_0_1_000416 0.5 1.0
m_0_1_000417 0.0 0.0
m_0_1_000418 0.0 0.0
m_0_1_000419 0.5 1.0
m_0_1_000420 0.5 1.0
m_0_1_000421 0.5 1.0
m_0_1_000422 0.333333333333 1.0
m_0_1_000423 0.0 0.0
m_0_1_000424 0.5 1.0
m_0_1_000425 0.5 1.0
m_0_1_000426 0.0 0.0
m_0_1_000427 0.0 0.0
m_0_1_000428 0.0 0.0
m_0_1_000429 0.0 0.0
m_0_1_000430 0.0 0.0
m_0_1_000431 0.5 1.0
m_0_1_000432 0.0 0.0
m_0_1_000433 0.0 0.0
m

m_0_2_000205 0.666666666667 1.0
m_0_2_000206 0.666666666667 1.0
m_0_2_000207 0.666666666667 1.0
m_0_2_000208 0.666666666667 1.0
m_0_2_000209 0.666666666667 1.0
m_0_2_000210 0.5 0.5
m_0_2_000211 0.666666666667 1.0
m_0_2_000212 0.666666666667 1.0
m_0_2_000213 0.666666666667 1.0
m_0_2_000214 0.666666666667 1.0
m_0_2_000215 0.666666666667 1.0
m_0_2_000216 0.666666666667 1.0
m_0_2_000217 0.666666666667 1.0
m_0_2_000218 0.666666666667 1.0
m_0_2_000219 0.5 0.5
m_0_2_000220 0.5 0.5
m_0_2_000221 0.666666666667 1.0
m_0_2_000222 0.666666666667 1.0
m_0_2_000223 0.666666666667 1.0
m_0_2_000224 0.666666666667 1.0
m_0_2_000225 0.666666666667 1.0
m_0_2_000226 1.0 0.5
m_0_2_000227 0.666666666667 1.0
m_0_2_000228 0.5 1.0
m_0_2_000229 0.666666666667 1.0
m_0_2_000230 1.0 0.5
m_0_2_000231 1.0 0.5
m_0_2_000232 0.666666666667 1.0
m_0_2_000233 0.333333333333 0.5
m_0_2_000234 0.666666666667 1.0
m_0_2_000235 0.666666666667 1.0
m_0_2_000236 0.5 0.5
m_0_2_000237 0.666666666667 1.0
m_0_2_000238 0.666666666667 1.0


m_0_2_000495 0.5 0.5
m_0_2_000496 0.666666666667 1.0
m_0_2_000497 0.666666666667 1.0
m_0_2_000498 0.666666666667 1.0
m_0_2_000499 0.666666666667 1.0
m_0_3_000000 0.75 1.0
m_0_3_000001 0.75 1.0
m_0_3_000002 0.666666666667 0.666666666667
m_0_3_000003 0.75 1.0
m_0_3_000004 0.75 1.0
m_0_3_000005 0.666666666667 0.666666666667
m_0_3_000006 0.75 1.0
m_0_3_000007 1.0 0.666666666667
m_0_3_000008 0.666666666667 0.666666666667
m_0_3_000009 1.0 0.666666666667
m_0_3_000010 1.0 0.666666666667
m_0_3_000011 0.666666666667 0.666666666667
m_0_3_000012 0.5 0.666666666667
m_0_3_000013 0.75 1.0
m_0_3_000014 0.75 1.0
m_0_3_000015 0.75 1.0
m_0_3_000016 1.0 0.666666666667
m_0_3_000017 1.0 0.666666666667
m_0_3_000018 0.5 0.666666666667
m_0_3_000019 0.75 1.0
m_0_3_000020 0.75 1.0
m_0_3_000021 0.6 1.0
m_0_3_000022 0.75 1.0
m_0_3_000023 0.75 1.0
m_0_3_000024 0.4 0.666666666667
m_0_3_000025 0.75 1.0
m_0_3_000026 0.666666666667 0.666666666667
m_0_3_000027 0.75 1.0
m_0_3_000028 0.75 1.0
m_0_3_000029 0.75 1.0
m_0_3_0

m_0_3_000287 0.666666666667 0.666666666667
m_0_3_000288 0.666666666667 0.666666666667
m_0_3_000289 0.5 0.666666666667
m_0_3_000290 0.75 1.0
m_0_3_000291 0.666666666667 0.666666666667
m_0_3_000292 0.666666666667 0.666666666667
m_0_3_000293 0.75 1.0
m_0_3_000294 0.5 0.666666666667
m_0_3_000295 0.6 1.0
m_0_3_000296 0.75 1.0
m_0_3_000297 0.75 1.0
m_0_3_000298 0.75 1.0
m_0_3_000299 0.75 1.0
m_0_3_000300 0.666666666667 0.666666666667
m_0_3_000301 0.6 1.0
m_0_3_000302 0.75 1.0
m_0_3_000303 0.5 0.666666666667
m_0_3_000304 0.75 1.0
m_0_3_000305 0.666666666667 0.666666666667
m_0_3_000306 0.5 0.666666666667
m_0_3_000307 0.75 1.0
m_0_3_000308 0.75 1.0
m_0_3_000309 0.666666666667 0.666666666667
m_0_3_000310 0.75 1.0
m_0_3_000311 0.75 1.0
m_0_3_000312 0.75 1.0
m_0_3_000313 0.75 1.0
m_0_3_000314 0.75 1.0
m_0_3_000315 0.75 1.0
m_0_3_000316 0.75 1.0
m_0_3_000317 0.75 1.0
m_0_3_000318 0.75 1.0
m_0_3_000319 0.75 1.0
m_0_3_000320 0.75 1.0
m_0_3_000321 0.5 0.666666666667
m_0_3_000322 0.75 1.0
m_0_3_000323 

In [52]:
(boxes,probs,cats) = nn.classify('./data/train_0/mis/images/m_0_3_000049.png', net, NN_PROB_THRESH) 

In [53]:
labels = read_label('./data/train_0/mis/labels/m_0_3_000049.txt')
label_boxes = [ kitti_2_box_format(l) for l in labels ]
ap, recall = average_precision(label_boxes, boxes, IOU_THRESH)

In [54]:
ap

0.75

In [55]:
recall

1.0

In [38]:
boxes

[array([ 584.81933594,  220.34779358,  129.04016113,  105.63809204], dtype=float32),
 array([ 589.07788086,  173.05867004,   62.38708496,   49.52755737], dtype=float32),
 array([ 635.69897461,  169.13327026,   56.59399414,   41.10015869], dtype=float32)]

In [42]:
gb

[[610.0, 145.0, 654.0, 177.0],
 [556.0, 144.0, 612.0, 192.0],
 [518.0, 179.0, 646.0, 265.0]]

In [56]:
net.close()

AttributeError: 'tuple' object has no attribute 'close'